# L1_L2_Syndigo Mapping ML Multi Label
  - module name: L1_L2_Syndigo Mapping ML Multi Label.ipynb
  - Purpose: Apply Scikit MultiLearn algorithm for Syndigo Level 1 & Level 2 classificatoion of Syndigo data  to build modelt 
  - ReEference: "Deep dive into multi-label classification..! (With detailed Case Study)"
    - https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff

In [13]:
def prepare_data_1():
    pgm = inspect.currentframe().f_code.co_name  
    start_time = time.time()

    
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    #print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    #print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    #print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    #print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    print(f'Before fillna syndigo_mapped.shape {syndigo_mapped.shape}') 
    syndigo_mapped[['Level 1','Level 2']] = syndigo_mapped[['Level 1','Level 2']].fillna('other')
    print(f'After  fillna syndigo_mapped.shape {syndigo_mapped.shape}') 
                                  
    return  pim_gtin_mapped

In [93]:
def prepare_data_2(samp_frac = 1, test_size = 0.3):
    global syndigo_mapped, level_id1_map, id1_level_map, leve_id2_map, id2_level_map 
    global A_train, A_test, B_train, B_test, X_train, X_test, y_train, y_test
    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name 
    print(f'Data preparation 2  for sample_frac = {samp_frac} and  test_size = {test_size}' )
    syndigo_mapped = syndigo_mapped_bkup.copy()
    whole_frac = 1
    print (f'Before sample syndigo_mapped.shape {syndigo_mapped.shape }')
    if samp_frac  < whole_frac :   syndigo_mapped = syndigo_mapped.sample(frac = samp_frac,  random_state=42)
    print (f'After sample syndigo_mapped.shape {syndigo_mapped.shape }')
    
    # Add l1 and l2 col to  syndigo_mapped
    level_id1_map = dict(zip(syndigo_mapped['Level 1'].fillna('Other').unique(), range(syndigo_mapped['Level 1'].fillna('Other').nunique())))
    id1_level_map = dict(zip(range(syndigo_mapped['Level 1'].fillna('Other').nunique()), syndigo_mapped['Level 1'].fillna('Other').unique()))
    level_id2_map = dict(zip(syndigo_mapped['Level 2'].fillna('Other').unique(), range(syndigo_mapped['Level 2'].fillna('Other').nunique())))
    id2_level_map = dict(zip(range(syndigo_mapped['Level 2'].fillna('Other').nunique()), syndigo_mapped['Level 2'].fillna('Other').unique()))
    data = {'l1':  syndigo_mapped['Level 1'].fillna('Other').map(level1_id_map),
            'l2':  syndigo_mapped['Level 2'].fillna('Other').map(level2_id_map)
           }
    df_y = pd.DataFrame(data)
    syndigo_mapped = pd.merge(syndigo_mapped, df_y, left_index=True, right_index=True)
    
    # create X_train, y_train, X_testr, and y_test
    vect = TfidfVectorizer(ngram_range = (1,2), max_features = 50000)
    A_train, A_test, B_train, B_test = train_test_split(syndigo_mapped.GTIN.tolist(), syndigo_mapped[['Level 1','Level 2']].values.tolist(), test_size=test_size,  random_state=42)
    train, test= train_test_split(syndigo_mapped, test_size= test_size,  random_state=42)
    X_train = vect.fit_transform(train.ITEM_SUBCOM_text) #scipy.sparse._csr.csr_matrix
    X_test = vect.fit_transform(test.ITEM_SUBCOM_text)
    y_train = train[['l1', 'l2']]
    y_test = test[['l1', 'l2']]
        
    end_time = time.time()   
    desc = f' Elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)
    return  

In [94]:
prepare_data_2()

Data preparation 2  for sample_frac = 1 and  test_size = 0.3
Before sample syndigo_mapped.shape (259085, 24)
After sample syndigo_mapped.shape (259085, 24)
  Elapse_time for "prepare_data_2". It took 9.213451 seconds - 0hh:0mm:9ss.
 start time: Aug 24 2023 15:10:49  end time:  Aug 24 2023 15:10:58


In [98]:
def proc_predict():
     start_time = time.time()
     pgm = inspect.currentframe().f_code.co_name   

     #A_train, A_test, B_train, B_test = train_test_split(syndigo_mapped.GTIN.tolist(), syndigo_mapped[['Level 1','Level 2']].values.tolist(), test_size=test_size,  random_state=42)
     
     #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size,  random_state=42)
     #lr_tf = LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'saga', n_jobs=-1)
     #lr_tf = LabelPowerset(LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'saga', n_jobs=-1))  
     lr_tf = LabelPowerset(LogisticRegression())

     
     print(f"{pgm} Training starts")
     lr_tf.fit(X_train.toarray(), y_train)
     preds = lr_tf.predict(X_test)
     print(f"type(preds) {type(preds)} len(preds) {len(preds)} \n preds[0:10] {preds[0:10]}")
     preds_lrtf = preds
     probs = lr_tf.predict_proba(X_test)
     print(f"type(probs) {type(probs)} len(probs) {len(probs)} \n probs[0:10] {probs[0:10]}")   
     preds_train = lr_tf.predict(X_train)
     print(f"type(preds_train) {type(preds_train)} len(preds_train) {len(preds_train)} \n preds_train[0:10] {preds_train[0:10]} ")   
     probs_train = lr_tf.predict_proba(X_train)
     print(f"type(probs_train) {type(probs_train)} len(probs_train) {len(probs_train)} \n probs_train[0:10] {probs_train[0:10]}")   
     #print(classification_report(y_test, preds_test,labels = lr_tf.classes_, target_names = [id2_level_map[i] for i in lr_tf.classes_]))
     
    
    
     #print("Accuracy = ",accuracy_score(y_test,preds))
     #print("Display TFIDF metrics")
     #print(classification_report(y_test, preds,labels = lr_tf.classes_, target_names = [id_l1_l2_map[i] for i in lr_tf.classes_]))
    
     print(f"Done Training ")
     end_time = time.time()   
     desc = f' Elapse_time for "{pgm}".' 
     elapse_time (  start_time, end_time, desc)
     return preds, preds_lrtf, probs, preds_train, probs_train, lr_tf 

In [99]:
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict()

proc_predict Training starts


MemoryError: Unable to allocate 67.6 GiB for an array with shape (181359, 50000) and data type float64

In [4]:
def validation(sample_frec, test_size):
     start_time = time.time()
     pgm = inspect.currentframe().f_code.co_name     
     print(f'Validation for sample_frec = {sample_frec} and  test_size = {test_size}' ) 
     print("Accuracy = ",accuracy_score(y_test,preds))
     print (f' f1_score_weighted = {f1_score(y_test, preds, average="weighted")}')
     print (f" f1_score_macro = {f1_score(y_test, preds, average='macro')}")
     print (f" f1_score_micro = {f1_score(y_test, preds, average='micro')}")
     # Compute the Multiclass ROC AUC score
     #multi_class = 'multinomial'
     #score = roc_auc_score(y_test, preds, multi_class= multi_class)
     #print(f"ROC AUC score for Multiclass= {multi_class}: {score:.2f}")

     print("Display TFIDF metrics")
     test_metrics = pd.DataFrame(classification_report(y_test, preds,labels = lr_tf.classes_, target_names = [id_l1_l2_map[i] for i in lr_tf.classes_],  output_dict= True)).T
     #test_metrics = pd.DataFrame(classification_report(results_test.Actuals, results_test.Predictions, output_dict= True)).T 
     #print(test_metrics)
     #Stest_metrics.to_csv('L1_L2_Test_Metrics_yue.csv')
     tab_name = f'L1_L2_Test_Metrics_sample_frec{sample_frec}_test_size{test_size}'
     print(f'tab_name = {tab_name}')  
    
     end_time = time.time()   
     desc = f' Elapse_time for "{pgm}".' 
     elapse_time (  start_time, end_time, desc)    
     return test_metrics, tab_name 
                               

In [5]:
def proc_append():
    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name     
    test_new_proobs = []
    
    #print(f'len(preds)  {len(preds)}')
    for i in range(len(preds)):
        test_new_proobs.append(probs[i][np.argsort(probs[i])][::-1][:1].tolist())
    test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]
    #print(f'test_new_proobs {test_new_proobs}')
    
    train_new_proobs = []
    
    #print(f'len(preds_train)  {len(preds_train)}')
    for i in range(len(preds_train)):
         test_new_proobs.append(probs_train[i][np.argsort(probs_train[i])][::-1][:1].tolist())
    train_new_proobs = [element for sublist in list(train_new_proobs) for element in sublist]
    #print(f'train_new_proobs {train_new_proobs}')  
    
    # process testLevels & trainLevelss 
    testLevels = []
    for j in y_test:
        testLevels.append([i for i in level2_id_map if level2_id_map[i]==j][0])
    #print(f'testLevels {testLevels [0:10]}')
    testLevelss = []
    for j in preds:
        testLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])
    #print(f'testLevelss {testLevelss [0:10]}')
    
    trainLevelss = []
    for j in preds_train:
        trainLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])
     
    end_time = time.time()   
    desc = f' Elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)    
    return  test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss

In [6]:
def proc_save():

    start_time = time.time()
    pgm = inspect.currentframe().f_code.co_name 
    print("---------------------------------------------\FINAL COUNTS:\n---------------------------------------------")
    print("1)", len(A_test + A_train))
    print("2)", len(['Test']*len(A_test) + ['Train']*len(A_train)))
    print("3)", len(B_test + B_train))
    print("4)", len(testLevels + B_train))
    print("5)", len(testLevelss + trainLevelss))
    print("6)", int(len(test_new_proobs + train_new_proobs)))
    
    data = {
        'GTIN' : A_test + A_train,
        'Source': ['Test']*len(A_test) + ['Train']*len(A_train),
        'Actual l1_l2' : B_test + B_train,
        'Actuals' : testLevels + B_train,
        'Predictions' : testLevelss + trainLevelss,
        'Scores' : test_new_proobs + train_new_proobs,
    
    }
    
    df = pd.DataFrame(data)
    df = df.merge(pim_gtin_mapped[['GTIN', 'ITM_ID', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
        'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
        'SUBCOM_DSC', 'VND_ECOM_DSC']], on='GTIN', how='left')
    df = df[['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
        'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
        'VND_ECOM_DSC', 'Source', 'Actual l1_l2', 'Actuals', 'Predictions', 'Scores']]
    filename = 'Syndigo_Mapping_L1_L2_ML.csv'
    df.to_csv(DBFO + filename, index=False)
    print(f"{filename}  --file created successfully")

    end_time = time.time()   
    desc = f' Elapse_time for "{pgm}".' 
    elapse_time (  start_time, end_time, desc)
    return

In [32]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
"""
  - resolve  "No module named 'skmultilearn'"
    - conda install -c condo-forge scikit-multilearn
    - pip install scikit-multilearn
"""
from skmultilearn.problem_transform import LabelPowerset

from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score, f1_score
import re, time, inspect
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

pd.options.display.float_format = '{:20,.2f}'.format 
pd.set_option("display.max_rows", None)

# load  function of 'elapse_time'
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))
global syndigo_mapped

In [6]:
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [15]:
pim_gtin_mapped = prepare_data_1()

Before fillna syndigo_mapped.shape (259085, 24)
After  fillna syndigo_mapped.shape (259085, 24)


In [11]:
syndigo_mapped_bkup = syndigo_mapped.copy()

In [53]:
X, y = prepare_data_2(samp_frac = 0.3)

 samp_frac = 0.3
Before sample syndigo_mapped.shape (77726, 24)
After sample syndigo_mapped.shape (23318, 24)
  Elapse_time for "prepare_data_2". It took 0.890644 seconds - 0hh:0mm:0ss.
 start time: Aug 23 2023 23:13:55  end time:  Aug 23 2023 23:13:56


In [55]:
# MemoryError: Unable to allocate 20.3 GiB for an array with shape (54408, 50000) and data type float64
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3)

Training starts for test_size = 0.3 


AttributeError: 'list' object has no attribute 'shape'

In [11]:

preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3)
test_metric_1, tab_1 =  validation(sample_frec = 0.3, test_size = 0.3)

 samp_frac = 0.3
Before sample syndigo_mapped.shape (259085, 25)
After sample syndigo_mapped.shape (77726, 25)
 syndigo_mapped .shape (77726, 25)
  Elapse_time for "prepare_data_2". It took 2.861276 seconds - 0hh:0mm:2ss.
 start time: Aug 23 2023 01:31:01  end time:  Aug 23 2023 01:31:04
Training starts for test_size = 0.3 
Done Training 
  Elapse_time for "proc_predict". It took 111.158615 seconds - 0hh:1mm:51ss.
 start time: Aug 23 2023 01:31:04  end time:  Aug 23 2023 01:32:55
Validation for sample_frec = 0.3 and  test_size = 0.3
Accuracy =  0.8790633845098207
 f1_score_weighted = 0.8742430204178655
 f1_score_macro = 0.46950650637609476
 f1_score_micro = 0.8790633845098208
Display TFIDF metrics
tab_name = L1_L2_Test_Metrics_sample_frec0.3_test_size0.3
  Elapse_time for "validation". It took 0.095665 seconds - 0hh:0mm:0ss.
 start time: Aug 23 2023 01:32:55  end time:  Aug 23 2023 01:32:56


In [12]:

preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.1)
test_metric_2, tab_2 =  validation(sample_frec = 0.3, test_size = 0.1)

Training starts for test_size = 0.1 
Done Training 
  Elapse_time for "proc_predict". It took 154.818257 seconds - 0hh:2mm:34ss.
 start time: Aug 23 2023 01:32:56  end time:  Aug 23 2023 01:35:30
Validation for sample_frec = 0.3 and  test_size = 0.1
Accuracy =  0.8815129293708993
 f1_score_weighted = 0.8772497941434809
 f1_score_macro = 0.541360092896292
 f1_score_micro = 0.8815129293708993
Display TFIDF metrics
tab_name = L1_L2_Test_Metrics_sample_frec0.3_test_size0.1
  Elapse_time for "validation". It took 0.057141 seconds - 0hh:0mm:0ss.
 start time: Aug 23 2023 01:35:30  end time:  Aug 23 2023 01:35:30


In [14]:
syndigo_mapped = syndigo_mapped_bkup.copy()

In [16]:
X, y, l1_l2_id_map, id_l1_l2_map = prepare_data_2(samp_frac = 1)
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3)
test_metric_3, tab_3 = validation(sample_frec = 1, test_size = 0.3)

 samp_frac = 1
Before sample syndigo_mapped.shape (259085, 25)
After sample syndigo_mapped.shape (259085, 25)
 syndigo_mapped .shape (259085, 25)
  Elapse_time for "prepare_data_2". It took 9.952945 seconds - 0hh:0mm:9ss.
 start time: Aug 23 2023 01:50:22  end time:  Aug 23 2023 01:50:32
Training starts for test_size = 0.3 
Done Training 
  Elapse_time for "proc_predict". It took 494.629706 seconds - 0hh:8mm:14ss.
 start time: Aug 23 2023 01:50:32  end time:  Aug 23 2023 01:58:46
Validation for sample_frec = 1 and  test_size = 0.3
Accuracy =  0.8902426472480252
 f1_score_weighted = 0.8880122013849431
 f1_score_macro = 0.45908703124835426
 f1_score_micro = 0.8902426472480252
Display TFIDF metrics
tab_name = L1_L2_Test_Metrics_sample_frec1_test_size0.3
  Elapse_time for "validation". It took 0.371622 seconds - 0hh:0mm:0ss.
 start time: Aug 23 2023 01:58:46  end time:  Aug 23 2023 01:58:47


In [17]:
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.1)
test_metric_4, tab_4 = validation(sample_frec = 1, test_size = 0.1)

Training starts for test_size = 0.1 
Done Training 
  Elapse_time for "proc_predict". It took 623.698597 seconds - 0hh:10mm:23ss.
 start time: Aug 23 2023 02:10:27  end time:  Aug 23 2023 02:20:51
Validation for sample_frec = 1 and  test_size = 0.1
Accuracy =  0.8931645374194296
 f1_score_weighted = 0.8907344356162481
 f1_score_macro = 0.5624715577166559
 f1_score_micro = 0.8931645374194297
Display TFIDF metrics
tab_name = L1_L2_Test_Metrics_sample_frec1_test_size0.1
  Elapse_time for "validation". It took 0.320256 seconds - 0hh:0mm:0ss.
 start time: Aug 23 2023 02:20:51  end time:  Aug 23 2023 02:20:52


In [21]:
#  Write classification report to excel
#  save() is going to remove, use close() instead
from pandas import ExcelWriter
from pandas import ExcelFile
excel_file = DBFO + 'L1_L2_Test_Metrics_cls_rpt.xlsx'

writer = pd.ExcelWriter(excel_file) 
tab_1 = 'L1_L2_TM_smp_frec0.3_tst_siz0.3'
tab_2 = 'L1_L2_TM_smp_frec0.3_tst_siz0.1'
tab_3 = 'L1_L2_TM_smp_frec1_tst_siz0.3'
tab_4 = 'L1_L2_TM_smp_frec1_tst_siz0.1'

test_metric_1.to_excel(writer,tab_1)
test_metric_2.to_excel(writer,tab_2)
test_metric_3.to_excel(writer,tab_3)
test_metric_4.to_excel(writer,tab_4)
writer.close()

C:\Users\iny2819\AppData\Local\Temp\ipykernel_24040\2099386414.py:16: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [ ]:
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.1)

In [ ]:
test_metric_1, tab_1 =  validation(sample_frec = 0.3, test_size = 0.1)

In [ ]:
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3)

In [ ]:
test_metric_2, tab_2 =  validation(sample_frec = 0.3, test_size = 0.3)

In [ ]:
preds, preds_lrtf, probs, preds_train, probs_train, lr_tf = proc_predict(test_size = 0.3)

In [ ]:
#dic_test={"Hospitality Supplies":"hospitality_spplies", "Tobacco Products":"bacco_products"}
#dic_test={"Food / Beverages":"food_beverages"}


    X_subset, y_subset, subset_df, level2_id_map =  prepare_data_2(level__1)
    preds, preds_lrtf, probs, preds_train, probs_train = proc_predict(level__1)
    test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss = proc_append()
    proc_save(level__1, filenamee) 

## wip

In [ ]:
syndigo_mapped[['Level 1', 'Level 2']].value_counts()

In [ ]:
syndigo_mapped_bkup = syndigo_mapped.copy()

In [ ]:
syndigo_mapped['l1_l2'] = syndigo_mapped['Level 1'] + ' + ' + syndigo_mapped['Level 2']
syndigo_mapped.head()

In [ ]:
df_l1_l2 = pd.DataFrame(syndigo_mapped[['Level 1', 'Level 2']].value_counts(dropna = False).reset_index())
df_l1_l2 

In [ ]:
y[0:10]


In [ ]:
syndigo_mapped['l1_l2']

In [ ]:
syndigo_mapped['l1_l2'].value_counts(dropna =  False)

In [ ]:
s = syndigo_mapped['l1_l2'].value_counts(dropna =  False)
s1 = s[s < 10]
s1

In [ ]:
data = {"count" : s1} 

df1 = pd.DataFrame(s, columns=["count"])   #NW Return enmpty DF
df1 = pd.DataFrame(data)
df1['count'].value_counts() 

In [ ]:
len(y)

In [ ]:
getnnz(X)

In [ ]:
 test_metric_2, tab_2 =  validation(sample_frec = 0.3, test_size = 0.3)

In [17]:
syndigo_mapped['Level 2'].value_counts(dropna =  False)

Grocery                                                     74863
Personal Care                                               22727
Beverages                                                   21240
Cosmetics & Fragrances                                      19935
Frozen Foods                                                13582
Dairy & Egg Products                                         9018
Vitamins & Dietary Supplements                               8928
Wine                                                         7114
Pet Supplies                                                 6296
Beer                                                         6123
Healthcare                                                   5729
Bakery / Deli                                                5333
Kitchen                                                      5048
Health Treatments                                            4940
Cleaning Chemicals                                           4483
Meat / Pou

In [20]:
syndigo_mapped[['Level 1', 'Level 2']].value_counts(dropna =  False).sort_index() 


Level 1                                   Level 2                                                 
Agricultural Equipment                    Tires                                                           2
                                          Vehicle Parts                                                   1
Apparel                                   Clothing                                                      863
                                          Footwear                                                      110
                                          Personal Accessories                                          237
                                          Safety Apparel                                                 73
Appliances                                Cooktops                                                        2
                                          Microwaves                                                      5
                                     

In [65]:
syndigo_mapped.shape

(23318, 24)

In [69]:
#vect_subset = TfidfVectorizer(ngram_range = (1,2), max_features = 100000)# from 1000000 to 100000
#X_subset = vect_subset.fit_transform(syndigo_mapped.ITEM_SUBCOM_text)
level_id1_map = dict(zip(syndigo_mapped['Level 1'].fillna('Other').unique(), range(syndigo_mapped['Level 1'].fillna('Other').nunique())))
id1_level_map = dict(zip(range(syndigo_mapped['Level 1'].fillna('Other').nunique()), syndigo_mapped['Level 1'].fillna('Other').unique()))
level_id2_map = dict(zip(syndigo_mapped['Level 2'].fillna('Other').unique(), range(syndigo_mapped['Level 2'].fillna('Other').nunique())))
id2_level_map = dict(zip(range(syndigo_mapped['Level 2'].fillna('Other').nunique()), syndigo_mapped['Level 2'].fillna('Other').unique()))
data = {'l1':  syndigo_mapped['Level 1'].fillna('Other').map(level1_id_map),
        'l2':  syndigo_mapped['Level 2'].fillna('Other').map(level2_id_map)
       }
df_y = pd.DataFrame(data)


In [80]:
level_id1_map = dict(zip(syndigo_mapped['Level 1'].fillna('Other').unique(), range(syndigo_mapped['Level 1'].fillna('Other').nunique())))
id1_level_map = dict(zip(range(syndigo_mapped['Level 1'].fillna('Other').nunique()), syndigo_mapped['Level 1'].fillna('Other').unique()))
level_id2_map = dict(zip(syndigo_mapped['Level 2'].fillna('Other').unique(), range(syndigo_mapped['Level 2'].fillna('Other').nunique())))
id2_level_map = dict(zip(range(syndigo_mapped['Level 2'].fillna('Other').nunique()), syndigo_mapped['Level 2'].fillna('Other').unique()))
data = {'l1':  syndigo_mapped['Level 1'].fillna('Other').map(level1_id_map),
        'l2':  syndigo_mapped['Level 2'].fillna('Other').map(level2_id_map)
       }
df_y = pd.DataFrame(data)
syndigo_mapped = pd.merge(syndigo_mapped, df_y, left_index=True, right_index=True)
print (f"{syndigo_mapped [['Level 1', 'l1', 'Level 2','l2']].head()}")

                 Level 1  l1  \
53931   Food / Beverages   0   
165551  Food / Beverages   0   
200103  Food / Beverages   0   
256925  Food / Beverages   0   
68030   Food / Beverages   0   

                                                  Level 2  l2  
53931   Meat / Poultry / Seafood / Meat Substitutes (P...   0  
165551                                      Bakery / Deli   1  
200103                                            Grocery   2  
256925                               Dairy & Egg Products   3  
68030                                             Grocery   2  


In [76]:
df3 = pd.merge(df1, df_y, left_index=True, right_index=True)
df3.head()

,Level 1,Level 2,l1,l2
53931,Food / Beverages,Meat / Poultry / Seafood / Meat Substitutes (P...,0,0
165551,Food / Beverages,Bakery / Deli,0,1
200103,Food / Beverages,Grocery,0,2
256925,Food / Beverages,Dairy & Egg Products,0,3
68030,Food / Beverages,Grocery,0,2


In [73]:
df1 = syndigo_mapped[['Level 1', 'Level 2']]
df1.head()

,Level 1,Level 2
53931,Food / Beverages,Meat / Poultry / Seafood / Meat Substitutes (P...
165551,Food / Beverages,Bakery / Deli
200103,Food / Beverages,Grocery
256925,Food / Beverages,Dairy & Egg Products
68030,Food / Beverages,Grocery


In [74]:
df_y.head()

,l1,l2
53931,0,0
165551,0,1
200103,0,2
256925,0,3
68030,0,2


In [75]:
level_id2_map.items()

dict_items([('Meat / Poultry / Seafood / Meat Substitutes (Perishable)', 0), ('Bakery / Deli', 1), ('Grocery', 2), ('Dairy & Egg Products', 3), ('Personal Care', 4), ('Vitamins & Dietary Supplements', 5), ('Candles & Accessories', 6), ('other', 7), ('Cigars', 8), ('Cleaning Chemicals', 9), ('Plumbing', 10), ('Beer', 11), ('Healthcare', 12), ('Cosmetics & Fragrances', 13), ('Construction & Modeling', 14), ('Frozen Foods', 15), ('Wine', 16), ('Gardening', 17), ('Health Treatments', 18), ('Cigarettes', 19), ('Spirits', 20), ('Pet Supplies', 21), ('Beverages', 22), ('Produce', 23), ('Air Treatment', 24), ('Holiday Decorations', 25), ('Kitchen', 26), ('Baby & Toddler Food', 27), ('Fluids, Chemicals, Compounds', 28), ('Hand Tools', 29), ('Drum & Barrel Parts / Accessories', 30), ('Printed Media', 31), ('Telephones & Communications', 32), ('Alcoholic Cordials & Syrups', 33), ('Special Occasion & Party Supplies', 34), ('Flavored Alcoholic Beverages', 35), ('Trash & Recycling', 36), ('General C

In [70]:
level_id1_map.items()

dict_items([('Food / Beverages', 0), ('Health & Beauty', 1), ('Home & Venue Decoration', 2), ('other', 3), ('Tobacco Products', 4), ('Cleaning & Janitorial', 5), ('Plumbing & Water Service', 6), ('Beer / Wine / Spirits', 7), ('Toys / Games / Hobbies', 8), ('Gardening & Outdoors', 9), ('Livestock & Pet Supplies', 10), ('Heating / Ventilation / Air Conditioning', 11), ('Kitchen & Bathroom', 12), ('Automotive', 13), ('Tools', 14), ('Material Handling', 15), ('Books & Videos', 16), ('Electronics', 17), ('Arts & Crafts', 18), ('Hardware', 19), ('Childcare', 20), ('Lighting & Fans', 21), ('Not classified', 22), ('Office Supplies', 23), ('Hospitality Supplies', 24), ('Appliances', 25), ('Apparel', 26), ('Sports & Outdoor Recreation Equipment', 27), ('Building Supplies', 28), ('Marine', 29), ('Electrical', 30), ('Power Sports', 31), ('Paints & Coatings', 32), ('Flooring', 33), ('Furniture', 34)])

In [ ]:
df_new = syndigo_mapped.merge(df_y, 
pd.merge(df1, df2, left_index=True, right_index=True)                              
y = df_y.values.tolist()

In [72]:
syndigo_mapped.columns

Index(['GTIN', 'Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5',
       'Level 6', 'Level 7', 'Level 8', 'Level 9', 'Level 10', 'ITM_ID',
       'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
       'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
       'VND_ECOM_DSC', 'ITEM_SUBCOM_text'],
      dtype='object')

In [47]:
y_l2   = syndigo_mapped['Level 2'].fillna('Other').map(level2_id_map)
#df_y['l1'] =  syndigo_mapped['Level 1'].fillna('Other').map(level1_id_map)

In [66]:
df_y.shape

(77726, 2)

In [49]:
data = {'l1':  syndigo_mapped['Level 1'].fillna('Other').map(level1_id_map),
        'l2':  syndigo_mapped['Level 2'].fillna('Other').map(level2_id_map)
       }
df_y = pd.DataFrame(data)

In [50]:
df_y.head()

,l1,l2
256488,0,0
3328,1,1
210628,2,2
47440,2,3
140713,3,4


In [51]:
df_y.values.tolist()

[[0, 0],
 [1, 1],
 [2, 2],
 [2, 3],
 [3, 4],
 [2, 5],
 [1, 6],
 [0, 0],
 [1, 6],
 [1, 7],
 [4, 8],
 [2, 2],
 [2, 2],
 [0, 9],
 [5, 10],
 [2, 11],
 [1, 6],
 [1, 1],
 [1, 1],
 [1, 1],
 [6, 12],
 [7, 13],
 [7, 13],
 [2, 14],
 [2, 2],
 [1, 6],
 [7, 13],
 [2, 14],
 [1, 6],
 [2, 2],
 [1, 7],
 [8, 15],
 [7, 16],
 [1, 7],
 [2, 2],
 [2, 2],
 [1, 1],
 [9, 17],
 [7, 18],
 [10, 19],
 [2, 14],
 [1, 7],
 [2, 2],
 [1, 7],
 [2, 14],
 [1, 6],
 [2, 14],
 [1, 1],
 [2, 2],
 [2, 2],
 [7, 13],
 [0, 0],
 [2, 2],
 [2, 2],
 [2, 2],
 [11, 20],
 [2, 11],
 [2, 2],
 [12, 21],
 [2, 11],
 [2, 14],
 [2, 22],
 [1, 23],
 [1, 1],
 [2, 2],
 [2, 2],
 [1, 6],
 [2, 11],
 [13, 24],
 [2, 2],
 [2, 2],
 [2, 2],
 [1, 7],
 [2, 22],
 [7, 25],
 [2, 14],
 [0, 26],
 [2, 2],
 [2, 5],
 [2, 11],
 [2, 2],
 [9, 17],
 [2, 11],
 [1, 1],
 [2, 2],
 [2, 5],
 [2, 2],
 [2, 2],
 [7, 25],
 [2, 2],
 [0, 27],
 [6, 12],
 [2, 11],
 [2, 11],
 [14, 28],
 [2, 3],
 [7, 25],
 [2, 11],
 [7, 13],
 [2, 5],
 [1, 7],
 [13, 29],
 [2, 11],
 [9, 17],
 [2, 11],
 [6

In [59]:
     """
      type(X_train): scipy.sparse._csr.csr_matrix
      type(y_train): list [[0, 2], [1, 4], [0, 2],....
     """
     test_size = 42

     A_train, A_test, B_train, B_test = train_test_split(syndigo_mapped.GTIN.tolist(), syndigo_mapped[['Level 1','Level 2']].values.tolist(), test_size=test_size,  random_state=42)
     
     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size,  random_state=42)
     #lr_tf = LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'saga', n_jobs=-1)
     #lr_tf = LabelPowerset(LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'saga', n_jobs=-1))  
     #lr_tf = LabelPowerset(LogisticRegression())

In [63]:
type(X_train)

scipy.sparse._csr.csr_matrix

In [61]:
y_train[0:10]

[[0, 2],
 [1, 4],
 [0, 2],
 [1, 4],
 [27, 110],
 [10, 21],
 [0, 22],
 [7, 11],
 [1, 13],
 [7, 11]]

In [64]:
syndigo_mapped_bkup.shape

(259085, 24)